In [1]:
import duckdb
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date

In [2]:
def create_con():
    con = duckdb.connect(database=':memory:')
    return con

def execute_local(con, query):
    result = con.execute(query).fetchdf()
    return result

con = create_con()
q = '''
        CREATE TABLE patients AS SELECT * FROM read_csv_auto('Data/synthetic/csv/patients.csv')
    '''
execute_local(con, q)
patients = execute_local(con, '''select * from patients''')

In [13]:
def date_to_age(birthdates):
    today = date.today()
    return birthdates.apply(
                lambda x: x.apply(
                    lambda y: int(today.year - y.year - ((today.month, today.day) < (y.month, y.day)))))

# Build the dataframe
df = patients.where(pd.notnull(patients), None)
df = df.where(pd.notna(df), None)

# Convert birthdate to age
age_col = date_to_age(df[['BIRTHDATE']])
df["AGE"] = age_col

# Rearange collumn order
cols = ['Id', 'BIRTHDATE', 'DEATHDATE', 'AGE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']
df = df[cols] 

# We only want alive people
df.drop(df[df['DEATHDATE']  == df['DEATHDATE']].index, inplace=True)
df.drop(['DEATHDATE'], axis=1, inplace=True)

# drop irrelevant collumns or collumns that contain too sensitive information
df.drop(['LAT', 'LON', 'SSN', 'DRIVERS', 'PASSPORT', 'HEALTHCARE_EXPENSES', 'ADDRESS', 'CITY', 'STATE', 'COUNTY'], axis=1, inplace=True)
df

,Id,BIRTHDATE,AGE,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ZIP,HEALTHCARE_COVERAGE
0,b8d8578a-3403-a5c5-6e44-19cae89e66e6,1996-09-18,25,Mr.,Josiah310,Marks830,None,None,None,white,nonhispanic,M,Lelystad Flevoland NL,8241,28196.83
1,6f196958-2892-9208-d039-3bc0da53e441,1967-11-10,54,Mrs.,Chanel977,Corkery305,None,Erdman779,M,white,nonhispanic,F,Lelystad Flevoland NL,8303,401884.43
2,16b4683a-7530-6f59-8b55-55c2ff94cb4a,1981-12-03,40,Mr.,Shelton25,Kirlin939,None,None,M,white,nonhispanic,M,Lelystad Flevoland NL,8321,54141.90
3,30043951-c45e-bd62-73bf-33416a271dea,1997-07-16,24,Ms.,Marcia467,Ryan260,None,None,None,white,nonhispanic,F,Lelystad Flevoland NL,3897,54159.71
4,03b2c661-cd30-40a7-2afd-d600d07533da,1966-01-09,56,Mr.,Michel472,Berge125,None,None,M,white,nonhispanic,M,Emmeloord Flevoland NL,8307,96259.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,122adea7-6afd-81cb-2162-185263fa7ce9,1956-03-21,66,Mr.,Antonia30,Breitenberg711,None,None,M,white,nonhispanic,M,Emmeloord Flevoland NL,8321,137154.59
532,ba345a55-3a6d-8884-fa60-41d41375d2fb,1988-12-24,33,Ms.,Trina350,Schmeler639,None,None,S,white,nonhispanic,F,Lelystad Flevoland NL,8211,949358.17
533,6cebf2c8-2017-4e8a-3bff-d15e62ba8a3b,1970-01-18,52,Mrs.,Leena55,Ebert178,None,Casper496,M,white,nonhispanic,F,Dronten Flevoland NL,8302,621971.69
534,6e3e5e33-c0bd-6916-7b6f-d671e1296a3d,1979-01-24,43,Mrs.,Cleopatra935,Schmitt836,None,Schimmel440,M,white,nonhispanic,F,Emmeloord Flevoland NL,8302,414181.63


In [4]:
def isKAnonymized(df, k):
    for index, row in df.iterrows():
        query = ' & '.join([f'{col} == {row[col]}' for col in df.columns])
        rows = df.query(query)
        if rows.shape[0] < k:
            return False
    return True

def validate_k_anonimity(df):
    count = 0
    for i in range(len(df)):
        for j in range(len(df)):
            if i == j:
                continue
            current = df.iloc[i]
            if distinguishable(current, df.iloc[j]):
                count = count + 1
    print(count)

def generalize(df, depths):
    return df.apply(lambda x: x.apply(lambda y: int(int(y/(10**depths[x.name]))*(10**depths[x.name]))))
    
def make_k_anonymous(df, row_preservation=0.5):
    pass




In [5]:
def generalize(df, depths):
    return df.apply(lambda x: x.apply(lambda y: int(int(y/(10**depths[x.name]))*(10**depths[x.name]))))


depths = {
    'ZIP': 2,
    'AGE': 1
}


output = generalize(df[['ZIP', 'AGE']], depths)


# output = pd.concat([output, df2[['BIRTHDATE']]], axis=1, join="inner")

isKAnonymized(output, 2)

False